In [1]:
!rm -rf ~/.julia

^C


In [2]:
!apt-get update
!apt-get install -y libgl1-mesa-glx libglapi-mesa libx11-xcb1 libxcb-dri3-0 libxcb-present0 libxcb-sync1 libxshd2 libxext6

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,384 kB]
0% [Waiting for headers] [3 InRelease 14.2 kB/129 kB 11%] [Waiting for headers]^C
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libxshd2


In [1]:
# Package Installation
using Pkg
Pkg.add([
    "DifferentialEquations",
    "Optimization",
    "OptimizationOptimJL",
    "UnicodePlots",
    "Statistics",
    "Random"
])

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed OrdinaryDiffEqRosenbrock ──────────── v1.9.0
   Installed OrdinaryDiffEqRKN ─────────────────── v1.1.0
   Installed ConsoleProgressMonitor ────────────── v0.1.2
   Installed DifferentialEquations ─────────────── v7.16.1
   Installed FastGaussQuadrature ───────────────── v1.0.2
   Installed Accessors ─────────────────────────── v0.1.42
   Installed OrdinaryDiffEqStabilizedRK ────────── v1.1.0
   Installed NonlinearSolve ────────────────────── v4.6.0
   Installed OrdinaryDiffEqPRK ─────────────────── v1.1.0
   Installed BoundaryValueDiffEqFIRK ───────────── v1.6.0
   Installed AlmostBlockDiagonals ──────────────── v0.1.10
   Installed OrdinaryDiffEqSDIRK ───────────────── v1.3.0
   Installed BoundaryValueDiffEqMIRK ───────────── v1.6.0
   Installed SciMLJacobianOperators ────────────── v0.1.2
   Installed Sundials_jll ──────────────────────── v5.2.2+0
   Installed MarchingCubes ─────

In [2]:
# Import Libraries
using DifferentialEquations, Optimization, OptimizationOptimJL, UnicodePlots, Statistics, Random

In [3]:
# Initialize Parameters and Generate Synthetic Data
N_days = 25
const S0 = 1.0
u0 = [0.99*S0, 0.01*S0, 0.0, 0.0, 0.0]  # [S, I, R, H, D]
tspan = (0.0, Float64(N_days))
t = range(tspan[1], tspan[2], length=N_days)
p_true = [0.85, 0.1, 0.05, 0.03]  # [β, γ, δ, η]

function SIRHD!(du, u, p, t)
    S, I, R, H, D = u
    β, γ, δ, η = p
    du[1] = -β*S*I
    du[2] = β*S*I - γ*I - δ*I
    du[3] = γ*I
    du[4] = δ*I - η*H
    du[5] = η*H
end

prob = ODEProblem(SIRHD!, u0, tspan, p_true)
true_sol = Array(solve(prob, TRBDF2(), saveat=t))

5×25 Matrix{Float64}:
 0.99  0.977175    0.951591    …  0.00590004  0.00544555  0.00508282
 0.01  0.0205229   0.0414218      0.0966636   0.0830568   0.0713347
 0.0   0.00153467  0.00465832     0.598291    0.607665    0.615722
 0.0   0.00075669  0.00227348     0.214767    0.212775    0.210196
 0.0   1.06442e-5  5.56811e-5     0.0843787   0.0910578   0.0976652

In [4]:
#  Plot Synthetic Data
p1 = lineplot(t, true_sol[1,:], title="Synthetic Data", name="S (Susceptible)", xlabel="Time (days)", ylabel="Population")
lineplot!(p1, t, true_sol[2,:], name="I (Infected)")
lineplot!(p1, t, true_sol[3,:], name="R (Recovered)")
lineplot!(p1, t, true_sol[4,:], name="H (Hospitalized)")
lineplot!(p1, t, true_sol[5,:], name="D (Deceased)")
display(p1)

                ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Synthetic Data⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀                 
                ┌────────────────────────────────────────┐                 
              1 │⠉⠒⠤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ S (Susceptible) 
                │⠀⠀⠀⠈⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ I (Infected)    
                │⠀⠀⠀⠀⠈⢢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ R (Recovered)   
                │⠀⠀⠀⠀⠀⠈⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ H (Hospitalized)
                │⠀⠀⠀⠀⠀⠀⠸⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ D (Deceased)    
                │⠀⠀⠀⠀⠀⠀⠀⢣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣀⠀⠀⠀⠀⠀⠀⠀│                 
                │⠀⠀⠀⠀⠀⠀⠀⠘⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⠤⠒⠊⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀│                 
   Population   │⠀⠀⠀⠀⠀⠀⠀⠀⢣⠀⡠⠒⠊⠢⡀⠀⠀⠀⠀⠀⢀⡠⠒⠊⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                 
                │⠀⠀⠀⠀⠀⠀⠀⠀⠈⡞⠀⠀⠀⠀⠈⠑⢄⠀⡠⠊⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                 
                │⠀⠀⠀⠀⠀⠀⠀⠀⡜⢱⠀⠀⠀⠀⠀⢀⠔⠛⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                 
                │⠀⠀⠀⠀⠀⠀⠀⡸⠀⠀⢇⠀⠀⢀⠔⠁⠀⠀⠀⠣⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                 
            

In [5]:
#  Define Neural Network and UDE Model
function nn_beta(t, p)
    W1 = reshape(p[1:3], 3, 1)
    b1 = reshape(p[4:6], 3)
    W2 = reshape(p[7:9], 1, 3)
    b2 = p[10]
    h = tanh.(W1 .* t .+ b1)
    return 1.0 / (1.0 + exp(-(sum(W2 .* h) + b2)))
end

function SIRHD_UDE!(du, u, p, t)
    S, I, R, H, D = u
    p_nn = p[1:10]
    γ, δ, η = p[11:13]
    β = nn_beta(t, p_nn)
    du[1] = -β*S*I
    du[2] = β*S*I - γ*I - δ*I
    du[3] = γ*I
    du[4] = δ*I - η*H
    du[5] = η*H
end

SIRHD_UDE! (generic function with 1 method)

In [6]:
# Initialize Parameters and Define Loss Function
Random.seed!(42)
p_nn0 = randn(10) * 0.01 .+ 0.5
p0_vec = [p_nn0; 0.1; 0.05; 0.03]
ude_prob = ODEProblem(SIRHD_UDE!, u0, tspan, p0_vec)

function loss_function(p)
    sol = solve(ude_prob, TRBDF2(), u0=u0, p=p, saveat=t)
    if any(x->any(isnan,x), sol.u) || length(sol.t) != length(t)
        return Inf
    end
    return sum(abs2, sol .- true_sol)
end

loss_function (generic function with 1 method)

In [7]:
# Train the Model
println("Training the model...")
opt_func = OptimizationFunction((x,p)->loss_function(x), Optimization.AutoForwardDiff())
opt_prob = OptimizationProblem(opt_func, p0_vec)
trained_params = solve(opt_prob, OptimizationOptimJL.LBFGS(), maxiters=50)
println("Training complete.")

trained_sol = solve(ude_prob, TRBDF2(), p=trained_params.u, saveat=t)

Training the model...
Training complete.


retcode: Success
Interpolation: 1st order linear
t: 25-element Vector{Float64}:
  0.0
  1.0416666666666667
  2.0833333333333335
  3.125
  4.166666666666667
  5.208333333333333
  6.25
  7.291666666666667
  8.333333333333334
  9.375
 10.416666666666666
 11.458333333333334
 12.5
 13.541666666666666
 14.583333333333334
 15.625
 16.666666666666668
 17.708333333333332
 18.75
 19.791666666666668
 20.833333333333332
 21.875
 22.916666666666668
 23.958333333333332
 25.0
u: 25-element Vector{Vector{Float64}}:
 [0.99, 0.01, 0.0, 0.0, 0.0]
 [0.9773064741132007, 0.02039952119423786, 0.00152871445540835, 0.00075464363017813, 1.0646606974986136e-5]
 [0.9518527467321651, 0.041191053805853355, 0.00463558017414561, 0.002264996501286169, 5.562278654973405e-5]
 [0.9031846728469676, 0.08058594918296516, 0.010815184810029806, 0.005245124042230887, 0.00016906911780671164]
 [0.8177506869753016, 0.14840474068677384, 0.022553871462367273, 0.010875313507359737, 0.00041538736819742684]
 [0.6876238127091835, 0.247

In [8]:
# Extrapolation and Plotting
N_days2 = 50
tspan2 = (0.0, Float64(N_days2))
t2 = range(tspan2[1], tspan2[2], length=N_days2)
ude_prob2 = ODEProblem(SIRHD_UDE!, u0, tspan2, trained_params.u)
trained_sol2 = solve(ude_prob2, TRBDF2(), saveat=t2)

p2 = lineplot(t2, trained_sol2[1,:], title="Extrapolated Solutions", name="S (Susceptible)", xlabel="Time (days)", ylabel="Population")
lineplot!(p2, t2, trained_sol2[2,:], name="I (Infected)")
lineplot!(p2, t2, trained_sol2[3,:], name="R (Recovered)")
lineplot!(p2, t2, trained_sol2[4,:], name="H (Hospitalized)")
lineplot!(p2, t2, trained_sol2[5,:], name="D (Deceased)")
display(p2)

                ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Extrapolated Solutions⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀                 
                ┌────────────────────────────────────────┐                 
              1 │⠑⢤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ S (Susceptible) 
                │⠀⠀⢣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ I (Infected)    
                │⠀⠀⠈⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ R (Recovered)   
                │⠀⠀⠀⢱⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ H (Hospitalized)
                │⠀⠀⠀⠈⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ D (Deceased)    
                │⠀⠀⠀⠀⢣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⡤⠤⠔⠒⠒⠒⠚⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉│                 
                │⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡠⠖⠊⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                 
   Population   │⠀⠀⠀⠀⠈⡆⡔⠓⢄⠀⠀⠀⢀⠔⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                 
                │⠀⠀⠀⠀⠀⣿⠀⠀⠈⢢⢀⠎⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                 
                │⠀⠀⠀⠀⢀⠿⡀⠀⠀⢠⠻⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                 
                │⠀⠀⠀⠀⡜⠀⡇⠀⡰⠁⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                 
            

In [9]:
# Parameter Uncertainty Estimation (Bootstrap)
function bootstrap_parameters(n_samples)
    parameter_samples = []
    for i in 1:n_samples
        println("Bootstrap sample $i of $n_samples")
        try
            noisy_data = true_sol .+ 0.01 .* randn(size(true_sol))
            function bootstrap_loss(p)
                sol = solve(ude_prob, TRBDF2(), u0=u0, p=p, saveat=t)
                if any(x->any(isnan,x), sol.u) || length(sol.t) != length(t)
                    return Inf
                end
                return sum(abs2, sol .- noisy_data)
            end
            opt_func = OptimizationFunction((x,p)->bootstrap_loss(x), Optimization.AutoForwardDiff())
            opt_prob = OptimizationProblem(opt_func, p0_vec)
            res = solve(opt_prob, OptimizationOptimJL.LBFGS(), maxiters=50)
            if !any(isnan, res.u)
                push!(parameter_samples, res.u)
            end
        catch e
            println("Error in bootstrap sample $i: ", e)
            continue
        end
    end
    return parameter_samples
end

println("Estimating parameter uncertainty (bootstrap)...")
parameter_samples = bootstrap_parameters(5)

if !isempty(parameter_samples)
    param_mat = reduce(hcat, parameter_samples)
    param_means = mean(param_mat, dims=2)
    param_stds = std(param_mat, dims=2)
    println("\nNumber of successful bootstrap samples: ", length(parameter_samples))
    println("Parameter means (first 5 shown): ", param_means[1:5])
    println("Parameter stds  (first 5 shown): ", param_stds[1:5])
else
    println("No successful bootstrap samples obtained.")
end

Estimating parameter uncertainty (bootstrap)...
Bootstrap sample 1 of 5
Bootstrap sample 2 of 5
Bootstrap sample 3 of 5
Bootstrap sample 4 of 5
Bootstrap sample 5 of 5

Number of successful bootstrap samples: 5
Parameter means (first 5 shown): [0.8478991382683716, 0.8410145713479251, 0.8427964158247535, 0.8658318165166241, 0.8668445539359011]
Parameter stds  (first 5 shown): [0.2770797454210431, 0.2672399711248466, 0.2727984056133083, 0.30972623736311544, 0.3015137421062973]


In [10]:
# Intervention Simulation
function SIRHD_UDE_intervention!(du, u, p, t)
    S, I, R, H, D = u
    p_nn = p[1:10]
    γ, δ, η = p[11:13]
    β = nn_beta(t, p_nn)
    if t > 20
        β *= 0.5  # 50% reduction after day 20
    end
    du[1] = -β*S*I
    du[2] = β*S*I - γ*I - δ*I
    du[3] = γ*I
    du[4] = δ*I - η*H
    du[5] = η*H
end

intervention_prob = ODEProblem(SIRHD_UDE_intervention!, u0, tspan2, trained_params.u)
intervention_sol = solve(intervention_prob, TRBDF2(), saveat=t2)

p3 = lineplot(t2, trained_sol2[2,:], title="Intervention Effects (Infected)", name="No intervention", xlabel="Time (days)", ylabel="Infected Population")
lineplot!(p3, t2, intervention_sol[2,:], name="With intervention")
display(p3)

max_no_int, day_no_int = findmax(trained_sol2[2,:])
max_int, day_int = findmax(intervention_sol[2,:])
println("No intervention: Peak infected = $(max_no_int) at day $(t2[day_no_int])")
println("With intervention: Peak infected = $(max_int) at day $(t2[day_int])")

                           ⠀⠀⠀⠀⠀⠀Intervention Effects (Infected)⠀⠀⠀⠀⠀                  
                           ┌────────────────────────────────────────┐                  
                       0.6 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ No intervention  
                           │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ With intervention
                           │⠀⠀⠀⠀⠀⠀⡔⠑⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                  
                           │⠀⠀⠀⠀⠀⢰⠁⠀⠘⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                  
                           │⠀⠀⠀⠀⠀⡜⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                  
                           │⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⢇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                  
                           │⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠘⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                  
   Infected Population     │⠀⠀⠀⠀⡸⠀⠀⠀⠀⠀⠀⠘⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                  
                           │⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠱⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                  
                           │⠀⠀⠀⢰

No intervention: Peak infected = 0.5188175777143174 at day 9.183673469387756
With intervention: Peak infected = 0.5188175777143174 at day 9.183673469387756


In [11]:
# Multiple Intervention Scenarios
function simulate_intervention_scenario(reduction_factor, start_day)
    function SIRHD_UDE_scenario!(du, u, p, t)
        S, I, R, H, D = u
        p_nn = p[1:10]
        γ, δ, η = p[11:13]
        β = nn_beta(t, p_nn)
        if t > start_day
            β *= reduction_factor
        end
        du[1] = -β*S*I
        du[2] = β*S*I - γ*I - δ*I
        du[3] = γ*I
        du[4] = δ*I - η*H
        du[5] = η*H
    end
    scenario_prob = ODEProblem(SIRHD_UDE_scenario!, u0, tspan2, trained_params.u)
    scenario_sol = solve(scenario_prob, TRBDF2(), saveat=t2)
    return scenario_sol
end

scenarios = [
    (0.7, 15, "30% reduction at day 15"),
    (0.5, 20, "50% reduction at day 20"),
    (0.3, 25, "70% reduction at day 25")
]

p4 = lineplot(t2, trained_sol2[2,:], title="Different Intervention Scenarios", name="No intervention", xlabel="Time (days)", ylabel="Infected Population")
for (red_factor, start_day, label) in scenarios
    scenario_sol = simulate_intervention_scenario(red_factor, start_day)
    lineplot!(p4, t2, scenario_sol[2,:], name=label)
    max_inf, day_inf = findmax(scenario_sol[2,:])
    println("$label: Peak infected = $(max_inf) at day $(t2[day_inf])")
end
display(p4)

30% reduction at day 15: Peak infected = 0.5188175777143174 at day 9.183673469387756
50% reduction at day 20: Peak infected = 0.5188175777143174 at day 9.183673469387756
70% reduction at day 25: Peak infected = 0.5188175777143174 at day 9.183673469387756


                           ⠀⠀⠀⠀⠀Different Intervention Scenarios⠀⠀⠀⠀⠀                        
                           ┌────────────────────────────────────────┐                        
                       0.6 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ No intervention        
                           │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 30% reduction at day 15
                           │⠀⠀⠀⠀⠀⠀⡔⠑⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 50% reduction at day 20
                           │⠀⠀⠀⠀⠀⢰⠁⠀⠘⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 70% reduction at day 25
                           │⠀⠀⠀⠀⠀⡜⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                        
                           │⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⢇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                        
                           │⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠘⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                        
   Infected Population     │⠀⠀⠀⠀⡸⠀⠀⠀⠀⠀⠀⠘⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│                        
                           │⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠱⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀